In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import os
import json
%matplotlib inline
import seaborn as sns
import datetime

import lightgbm as lgb
import xgboost as xgb


from scipy.optimize import minimize
from bayes_opt import BayesianOptimization

from sklearn import preprocessing
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import gc


In [3]:
#col = ['transactionRevenue_size','hits_sum','hits_min','hits_max','hits_mean','hits_median','pageviews_sum','pageviews_min','pageviews_max','pageviews_mean','pageviews_median','bounces_sum','bounces_mean','bounces_median','newVisits_sum','newVisits_mean','newVisits_median','channelGrouping_min','channelGrouping_max','channelGrouping_mean','channelGrouping_median','channelGrouping_std','channelGrouping_var','socialEngagementType_min','socialEngagementType_max','socialEngagementType_mean','socialEngagementType_median','socialEngagementType_std','socialEngagementType_var','browser_min','browser_max','browser_mean','browser_median','browser_std','browser_var','deviceCategory_min','deviceCategory_max','deviceCategory_mean','deviceCategory_median','deviceCategory_std','deviceCategory_var','operatingSystem_min','operatingSystem_max','operatingSystem_mean','operatingSystem_median','operatingSystem_std','operatingSystem_var','city_min','city_max','city_mean','city_median','city_std','city_var','continent_min','continent_max','continent_mean','continent_median','continent_std','continent_var','country_min','country_max','country_mean','country_median','country_std','country_var','metro_min','metro_max','metro_mean','metro_median','metro_std','metro_var','networkDomain_min','networkDomain_max','networkDomain_mean','networkDomain_median','networkDomain_std','networkDomain_var','region_min','region_max','region_mean','region_median','region_std','region_var','subContinent_min','subContinent_max','subContinent_mean','subContinent_median','subContinent_std','subContinent_var','medium_min','medium_max','medium_mean','medium_median','medium_std','medium_var','source_min','source_max','source_mean','source_median','source_std','source_var','sess_date_dow_min','sess_date_dow_max','sess_date_dow_mean','sess_date_dow_median','sess_date_dow_std','sess_date_dow_var','sess_date_doy_min','sess_date_doy_max','sess_date_doy_mean','sess_date_doy_median','sess_date_doy_std','sess_date_doy_var','sess_date_mon_min','sess_date_mon_max','sess_date_mon_mean','sess_date_mon_median','sess_date_mon_std','sess_date_mon_var','sess_date_week_min','sess_date_week_max','sess_date_week_mean','sess_date_week_median','sess_date_week_std','sess_date_week_var','date_diff']
#test_users[['transactionRevenue_size','hits_sum','hits_min','hits_max','hits_mean','hits_median','pageviews_sum','pageviews_min','pageviews_max','pageviews_mean','pageviews_median','bounces_sum','bounces_mean','bounces_median','newVisits_sum','newVisits_mean','newVisits_median','channelGrouping_min','channelGrouping_max','channelGrouping_mean','channelGrouping_median','channelGrouping_std','channelGrouping_var','socialEngagementType_min','socialEngagementType_max','socialEngagementType_mean','socialEngagementType_median','socialEngagementType_std','socialEngagementType_var','browser_min','browser_max','browser_mean','browser_median','browser_std','browser_var','deviceCategory_min','deviceCategory_max','deviceCategory_mean','deviceCategory_median','deviceCategory_std','deviceCategory_var','operatingSystem_min','operatingSystem_max','operatingSystem_mean','operatingSystem_median','operatingSystem_std','operatingSystem_var','city_min','city_max','city_mean','city_median','city_std','city_var','continent_min','continent_max','continent_mean','continent_median','continent_std','continent_var','country_min','country_max','country_mean','country_median','country_std','country_var','metro_min','metro_max','metro_mean','metro_median','metro_std','metro_var','networkDomain_min','networkDomain_max','networkDomain_mean','networkDomain_median','networkDomain_std','networkDomain_var','region_min','region_max','region_mean','region_median','region_std','region_var','subContinent_min','subContinent_max','subContinent_mean','subContinent_median','subContinent_std','subContinent_var','medium_min','medium_max','medium_mean','medium_median','medium_std','medium_var','source_min','source_max','source_mean','source_median','source_std','source_var','sess_date_dow_min','sess_date_dow_max','sess_date_dow_mean','sess_date_dow_median','sess_date_dow_std','sess_date_dow_var','sess_date_doy_min','sess_date_doy_max','sess_date_doy_mean','sess_date_doy_median','sess_date_doy_std','sess_date_doy_var','sess_date_mon_min','sess_date_mon_max','sess_date_mon_mean','sess_date_mon_median','sess_date_mon_std','sess_date_mon_var','sess_date_week_min','sess_date_week_max','sess_date_week_mean','sess_date_week_median','sess_date_week_std','sess_date_week_var','date_diff']].to_csv('test_users_mine.csv')
#usecols=['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId','visitNumber','visitStartTime','totals', 'device', 'geoNetwork', 'socialEngagementType', 'trafficSource'],

def load_df(type='train',csv_path='input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str',
                           'date': 'str'
                           }, # Important!!
                     usecols=['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId','visitNumber','visitStartTime','totals', 'device', 'geoNetwork', 'socialEngagementType', 'trafficSource'],
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    df.fillna(0,inplace=True)
    
    if (type == 'test'):
        df['totals.transactionRevenue'] = 0
    
    print("Loaded CSV For : {} - Shape : {}".format(type,df.shape))
    return df



def date_feature(type,input_df='train'):
    date_feat = ['sess_date_hours','sess_date_dom','sess_date_dow','sess_date_doy','sess_date_mon','sess_date_week']
    
    input_df['date']=pd.to_datetime(input_df['date'])
    input_df[date_feat[0]] = input_df['date'].dt.hour
    input_df[date_feat[1]] = input_df['date'].dt.day
    input_df[date_feat[2]] = input_df['date'].dt.dayofweek
    input_df[date_feat[3]] = input_df['date'].dt.dayofyear
    input_df[date_feat[4]] = input_df['date'].dt.month
    input_df[date_feat[5]] = input_df['date'].dt.weekofyear
    
    print ("Date Feature Completed For: {} - Shape:{}".format(type,input_df.shape))
    return input_df,date_feat


def factorize_categorical (type='train',input_df='train',cat_indexer=None):
    cat_feat=['channelGrouping','socialEngagementType',
              'device.browser','device.deviceCategory',
              'device.operatingSystem','geoNetwork.city',
              'geoNetwork.continent','geoNetwork.country',
              'geoNetwork.metro','geoNetwork.networkDomain',
              'geoNetwork.region','geoNetwork.subContinent',
              'trafficSource.adContent','trafficSource.campaign', 
              'trafficSource.isTrueDirect','trafficSource.keyword', 
              'trafficSource.medium','trafficSource.referralPath', 
              'trafficSource.source']

    # 'trafficSource.adContent','trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.gclId'
    # 'trafficSource.adwordsClickInfo.isVideoAd','trafficSource.adwordsClickInfo.page','trafficSource.adwordsClickInfo.slot'
    # 'trafficSource.campaign','trafficSource.isTrueDirect','trafficSource.keyword','trafficSource.referralPath'
    
    num_feat = ['totals.bounces','totals.hits','totals.newVisits','totals.pageviews','totals.transactionRevenue']
    
    if (cat_indexer is None):
        cat_indexer = {}
        for col in cat_feat:
            #print ("Factorizing Column: {}".format(col))
            input_df[col],indexer = pd.factorize(input_df[col])
            cat_indexer[col]=indexer
    else:
        for col in cat_feat:
            #print ("Factorizing Column: {}".format(col))
            input_df[col] = cat_indexer[col].get_indexer(input_df[col])
        
    for col in num_feat:
        #if ( (type == 'train') or (type == 'test' and col != 'totals.transactionRevenue') ):
        #print ("Converting Column to float:{}".format(col))
        input_df[col]=input_df[col].astype(float)
            
    #if (type == 'train'):
    input_df['totals.transactionRevenue'].fillna(0,inplace=True)
    
    print ("Factorize Completed For: {} - Shape:{}".format(type,input_df.shape))
    
    return input_df,cat_feat,num_feat,cat_indexer


def summrize_by_user(type='train',input_df='train',cat_feat=None,num_feat=None,date_feat=None):
    aggs = {}
    y = []
    
    aggs = {
        'date': ['min', 'max'],
        'totals.transactionRevenue': ['sum', 'size'],
        'totals.hits': ['sum', 'min', 'max', 'mean', 'median'],
        'totals.pageviews': ['sum', 'min', 'max', 'mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }
        
    #if (type == 'train'):
        #aggs['totals.transactionRevenue'] = ['sum','size']
    
    """"
    for num_col in num_feat:
        if (num_col != 'totals.transactionRevenue'):
            aggs[num_col] = ['sum','min','max','mean','median']
    """
    
    for cat_col in cat_feat:
        aggs[cat_col] = ['min','max','mean','median','std','var']
        
    for date_col in date_feat:
        aggs[date_col] = ['min','max','mean','median','std','var']
        
    users=input_df.groupby('fullVisitorId').agg(aggs)
    
    new_col = [l1+'_'+l2 for l1 in aggs.keys() for l2 in aggs[l1]]
    users.columns = new_col
    
    #if (type == 'train'):
    users['totals.transactionRevenue_sum'] = np.log1p(users['totals.transactionRevenue_sum'])
    y = users['totals.transactionRevenue_sum']
        #users.drop(['totals.transactionRevenue_sum'],axis=1,inplace=True)
    
    users['date_diff'] = (users.date_max - users.date_min).astype(np.int64) // (24 * 3600 * 1e9)
    
    users.drop(['date_min','date_max','totals.transactionRevenue_sum'], axis=1, inplace=True)

    print ("Aggregation Completed For:{} - Shape:{}".format(type,users.shape))
    
    return users,y



def get_KFold(n_splits=5,shuffle=True,random_state=7):
    KFolds = KFold(n_splits=n_splits,shuffle=True,random_state=7)
    
    #folds=KFolds.split(df)
    
    return KFolds



def get_GroupKFolds(df=None, n_splits=5):
    """Returns dataframe indices corresponding to Visitors Group KFold"""
    # Get sorted unique visitors
    unique_vis = np.array(sorted(df['fullVisitorId'].unique()))

    # Get folds
    folds = GroupKFold(n_splits=n_splits)
    fold_ids = []
    ids = np.arange(df.shape[0])
    for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
        fold_ids.append(
            [
                ids[df['fullVisitorId'].isin(unique_vis[trn_vis])],
                ids[df['fullVisitorId'].isin(unique_vis[val_vis])]
            ]
        )

    return fold_ids



def rf_kfold(train_X,train_y,test_X,n_estimators=20000,max_depth=12,verbose=2,n_splits=10):
    print ("Train & Predict using Random Forest with Splits:{}".format(n_splits))
    val_pred = np.zeros((train_X.shape[0]))
    test_pred = np.zeros((test_X.shape[0]))
    
    rf_feature_importance_df = pd.DataFrame()

    Folds = KFold(n_splits=n_splits,shuffle=True,random_state=7)

    params = {
        'n_estimators': n_estimators,
        'criterion' : 'mse',
        'max_depth' : max_depth ,
        'random_state': 123,
        'verbose': verbose
    }

    for fold_,(train_,val_) in enumerate(Folds.split(train_X)):
        rf = RandomForestRegressor()
        rf.set_params(**params)
        
        X = train_X.iloc[train_]
        y = train_y.iloc[train_]
        rf.fit(X,y)
        
        val_pred[val_]=rf.predict(train_X.iloc[val_])
        
        curr_test_pred=rf.predict(test_X)
        curr_test_pred[curr_test_pred < 0] = 0
        test_pred += curr_test_pred / n_splits
        print ("Fold: {}   RMSE:{}".format(fold_,mean_squared_error(val_pred[val_],train_y[val_])**0.5) )
        
        fold_feature_importance_df = pd.DataFrame()
        fold_feature_importance_df["feature"] = train_X.columns
        fold_feature_importance_df["importance"] = rf.feature_importances_
        fold_feature_importance_df["fold"] = fold_ + 1
        rf_feature_importance_df = pd.concat([rf_feature_importance_df,fold_feature_importance_df],axis=0)
        
    val_pred[val_pred<0] = 0
    
    print ("Fold: A   RMSE:{} ".format(mean_squared_error(val_pred,train_y)**0.5) )
    
    return val_pred,test_pred,rf_feature_importance_df



def lgb_kfold(train_X=None,train_y=None,test_X=None,features=None,learning_rate=0.01,n_estimators=20000,num_leaves=128,
              subsample=0.2217,colsample_bytree=0.6810,min_split_gain=np.power(10.0, -4.9380),
              reg_alpha=np.power(10.0, -3.2454),reg_lambda=np.power(10.0, -4.8571),min_child_weight=np.power(10.0, 2),
              early_stopping_rounds=100,verbose=False,foldType='Group',n_splits=10):
    
    print ("Train & Predict using Light GBM with Splits:{}".format(n_splits))
    
    val_pred = np.zeros((train_X.shape[0]))
    test_pred = np.zeros((test_X.shape[0]))
    
    lgb_feature_importance_df = pd.DataFrame()
    lgb_validation_loss_df = pd.DataFrame()
    
    
    if (foldType == 'Group'):
        Folds = get_GroupKFolds(df=train_X, n_splits=n_splits)
        folds = Folds
        
    else:
        Folds = get_KFold(n_splits=n_splits,shuffle=True,random_state=7)
        folds = Folds.split(train_X)
    
    
    params = {
        'learning_rate': learning_rate,
        'n_estimators': n_estimators,
        'num_leaves': num_leaves,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'min_split_gain': min_split_gain,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'min_child_weight': min_child_weight,
        'silent': True
    }
        
    for fold_,(train_,val_) in enumerate(folds):
        lgb_model = lgb.LGBMRegressor(**params)
        
        lgb_model.fit(
            X=train_X[features].iloc[train_],
            y=train_y.iloc[train_],
            eval_set=[(train_X[features].iloc[train_],train_y.iloc[train_]),(train_X[features].iloc[val_],train_y.iloc[val_])],
            eval_metric='rmse',
            early_stopping_rounds=early_stopping_rounds,
            verbose=verbose
        )
            
        val_pred[val_]=lgb_model.predict(train_X[features].iloc[val_])
        curr_test_pred=lgb_model.predict(test_X[features])
        curr_test_pred[curr_test_pred < 0] = 0
        test_pred += curr_test_pred / n_splits
        print ("Fold: {}   RMSE:{}   Best Iteration:{}".format(fold_,mean_squared_error(val_pred[val_],train_y[val_])**0.5,lgb_model.best_iteration_) )
        
        fold_feature_importance_df = pd.DataFrame()
        fold_feature_importance_df["feature"] = train_X[features].columns
        fold_feature_importance_df["importance"] = lgb_model.feature_importances_
        fold_feature_importance_df["fold"] = fold_ + 1
        lgb_feature_importance_df = pd.concat([lgb_feature_importance_df,fold_feature_importance_df],axis=0)
        
        lgb_fold_validation_loss_df = pd.DataFrame()
        lgb_fold_validation_loss_df['rmse']= lgb_model.evals_result_['valid_1']['rmse']
        lgb_fold_validation_loss_df['fold'] = fold_+1
        lgb_validation_loss_df = pd.concat([lgb_fold_validation_loss_df,lgb_validation_loss_df],axis=0)
        
        
    val_pred[val_pred<0] = 0
    
    print ("Fold: A   RMSE:{}   Best Iteration:None".format(mean_squared_error(val_pred,train_y)**0.5) )
    
    return val_pred,test_pred,lgb_feature_importance_df,lgb_validation_loss_df


    
def xgb_kfold(train_X,train_y,test_X,n_splits=10):
    print ("Train & Predict using XGBOOST with Splits:{}".format(n_splits))
    
    feature_importance_df = pd.DataFrame()
    
    Folds = KFold(n_splits=n_splits,shuffle=True,random_state=7)
    
    xgb_model = xgb.XGBRegressor(
        max_depth=12,
        learning_rate=0.003,
        booster='gbtree',
        objective='reg:linear',
        reg_lambda=0.8,
        min_child_weight=4,
        subsample=0.3,
        colsample_bytree=0.5,
        n_estimators=20000,
        silent=True,
        seed=239)
    
    val_pred = np.zeros((train_X.shape[0]))
    test_pred = np.zeros((test_X.shape[0]))

    for fold_,(train_,val_) in enumerate(Folds.split(train_X)):
        xgb_model.fit(
            X = train_X.iloc[train_],
            y = train_y[train_],
            eval_metric='rmse',
            eval_set=[(train_X.iloc[train_],train_y[train_]),(train_X.iloc[val_],train_y[val_])],
            early_stopping_rounds=50,
            verbose=200
        )

        val_pred[val_]=xgb_model.predict(train_X.iloc[val_])
        curr_test_pred=xgb_model.predict(test_X)
        curr_test_pred[curr_test_pred < 0] = 0
        test_pred += curr_test_pred / n_splits
        print ("Fold: {}   RMSE:{}   Best Iteration:{}".format(fold_,mean_squared_error(val_pred[val_],train_y[val_])**0.5,xgb_model.best_iteration) )
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = train_X.columns
        fold_importance_df["importance"] = xgb_model.feature_importances_
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df,fold_importance_df],axis=0)

        
    val_pred[val_pred<0] = 0
    
    print ("Fold: A   RMSE:{}   Best Iteration:None".format(mean_squared_error(val_pred,train_y)**0.5) )

    return val_pred,test_pred,feature_importance_df



def rmse_min_func(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    for weight, prediction in zip(weights, blend_train):
        final_prediction += weight*prediction
        
    return np.sqrt(mean_squared_error(trn_y,final_prediction))



def display_loss(model_validation_loss_df=None,title='Boosting Iteration Vs RMSE'):
    print ("Displaying Metric Loss")
    plt.figure(figsize=(18,12))
    plt.title(title)
    plt.xlabel('Boosting Iteration')
    plt.ylabel('RMSE')
    for k in np.arange(len(model_validation_loss_df['fold'].value_counts())):
        plt.plot(np.arange(len(model_validation_loss_df[model_validation_loss_df['fold'] == k+1]['rmse'])),model_validation_loss_df[model_validation_loss_df['fold'] == k+1]['rmse'],label='Fold {}'.format(k))
    
    plt.legend()
        


def display_feature_importance(feature_importance_df_=None,modelName="LightGBM Feature Importance (Over Folds)"):
    print ("Displaying Feature Importance")
    
    cols = feature_importance_df_[['feature','importance']].groupby(['feature']).mean().sort_values(
        by='importance',ascending=False)[:50].index
    
    best_features = feature_importance_df_[feature_importance_df_['feature'].isin(cols)]
    
    plt.figure(figsize=(12,20))
    sns.barplot(x="importance",y="feature",data=best_features.sort_values(by='importance',ascending=False))
    plt.title(modelName)
    plt.tight_layout()
    plt.savefig('figure/lgbm_feature_importance.png')
    
    
    
def write_submission_file(test_X=None,test_pred=None,filename=None):
    print ("Write the Submission File")
    
    test_X['PredictedLogRevenue'] = test_pred
    print ("Start Creating Submission File")
    
    test_X[['PredictedLogRevenue']].to_csv("submission/{}_{}.csv".format(filename,datetime.datetime.now().strftime('%Y%m%d_%H%M%S')))
    
    test_X.drop('PredictedLogRevenue',axis=1,inplace=True)

    return 'Success'


def blending_weights(blend_train=None):
    blend_train = []
    blend_test = []
    
    xgboost_val_pred.columns = ['val_pred']
    lgboost_val_pred.columns = ['val_pred']
    xgboost_test_pred.columns = ['val_pred']
    lgboost_test_pred.columns = ['val_pred']
    
    blend_train.append(xgboost_val_pred['val_pred'].values)
    blend_train.append(lgboost_val_pred['val_pred'].values)
    
    blend_test.append(xgboost_test_pred['val_pred'].values)
    blend_test.append(lgboost_test_pred['val_pred'].values)
    
    
    #Run the minimize function for a certain number
    print('\nFinding Blending Weights ...')
    res_list = []
    weights_list = []
    num_rounds_ensemble = 5000
    
    for k in range (num_rounds_ensemble):
        
        starting_values = np.random.uniform(size=len(blend_train))
        
    
        #######
        # https://www.kaggle.com/tilii7/cross-validation-weighted-linear-blending-errors
        # I used to think that weights should not be negative - many agree with that.
        # I've come around on that issues as negative weights sometimes do help.
        # If you don't think so, just swap the two lines below.
        #######

        #    bounds = [(0, 1)]*len(blend_train)
        bounds = [(-1, 1)] * len(blend_train)

        res = minimize(rmse_min_func,
                       starting_values,
                       method='L-BFGS-B',
                       bounds=bounds,
                       options={'disp': False,
                                'maxiter': 100000})
        res_list.append(res['fun'])
        weights_list.append(res['x'])
        print ("Iteration:{}\tScore:{}\tWeights:{}".format(k+1,res['fun'],([str(item) for item in res['x']])))

        bestSC = np.min(res_list)
        bestWght = weights_list[np.argmin(res_list)]
        weights = bestWght
        blend_score = round(bestSC, 6)

        print('\nEnsemble Score: {best_score}'.format(best_score=bestSC))
        print('\nBest Weights: {weights}'.format(weights=bestWght))

        test_ensemble = np.zeros(len(blend_test[0]))
        for k in range(len(blend_test)):
            print("Model: {} * Weight:{}".format(k+1, weights[k]))
            test_ensemble += blend_test[k] * weights[k]
    
    
def bayes_parameter_opt_lgb(X,y,num_boost_round=20000,nfold=5,early_stopping_rounds=100,metrics='rmse',random_state=1319):
    train_data = lgb.Dataset(data=X, label=y)
    
    '''Function for Light GBM Evaluate Function'''
    def lgb_eval(num_leaves,
                 max_depth,
                 subsample,
                 colsample_bytree,
                 min_split_gain,
                 reg_alpha,
                 reg_lambda,
                 min_child_weight
                 ):
    
        params = {}

        params["num_leaves"] = round(num_leaves)
        params['max_depth'] = round(max_depth)
        params['subsample'] = subsample
        params['colsample_bytree'] = colsample_bytree
        params['min_split_gain']=min_split_gain
        params['reg_alpha'] = reg_alpha
        params['reg_lambda'] = reg_lambda
        params['min_child_weight'] = min_child_weight


        cv_result = lgb.cv(params,
                           train_set=train_data,
                           num_boost_round=num_boost_round,
                           nfold=nfold,
                           metrics=metrics,
                           early_stopping_rounds=early_stopping_rounds,
                           verbose_eval=1,
                           seed=random_state)

        return max(cv_result['auc-mean'])
    '''End of Light GBM Evaluate Function'''
    
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'max_depth':(8,12),
                                            'subsample':(0.1,0.4),
                                            'colsample_bytree': (0.1,0.8),
                                            'min_split_gain': (0.001, 0.1),                                            
                                            'feature_fraction': (0.1, 0.9),
                                            'reg_alpha': (0.0001, 0.0008),
                                            'reg_lambda': (0.00001, 0.00008),
                                            'min_child_weight': (5, 200)}, random_state=0)
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    return lgbBO.res['max']['max_params']    


def lgb_evaluate(num_leaves,max_depth,subsample,colsample_bytree,min_split_gain,reg_alpha,reg_lambda,min_child_weight):

    params = {}

    params["num_leaves"] = int(round(num_leaves))
    #params["feature_fraction"] = round(feature_fraction)
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample_bytree
    params['min_split_gain']=min_split_gain
    params['reg_alpha'] = reg_alpha
    params['reg_lambda'] = reg_lambda
    params['min_child_weight'] = min_child_weight
    
    cv_result = lgb.cv(params,
                       train_set=train_data,
                       num_boost_round=num_boost_round,
                       nfold=nfold,
                       stratified=False,
                       metrics=metrics,
                       early_stopping_rounds=early_stopping_rounds,
                       verbose_eval=200,
                       seed=random_state
                       )

    return max(cv_result['rmse-mean'])


    lgbBO = BayesianOptimization(lgb_evaluate, {'num_leaves': (24, 45),
                                            'max_depth':(8,12),
                                            'subsample':(0.1,0.4),
                                            'colsample_bytree': (0.1,0.8),
                                            'min_split_gain': (0.001, 0.1),                                            
                                            'reg_alpha': (0.0001, 0.0008),
                                            'reg_lambda': (0.00001, 0.00008),
                                            'min_child_weight': (5, 200)}, random_state=0)




    lgbBO.maximize(init_points=init_round, n_iter=opt_round)

    



In [4]:
"""
def main():
    cat_Feature=[]
    num_feature=[]
    
    train = load_df('Train','input/train.csv')
    test  = load_df('Test','input/test.csv')

    train,date_feat = date_feature('Train',train)
    test,_  = date_feature('Test',test)    
    
    train,cat_feat,num_feat = factorize_categorical('Train',train)
    test,_,_  = factorize_categorical('Test',test)
    
    trn_users,trn_y = summrize_by_user('Train',train,cat_feat,num_feat,date_feat)
    test_users,_    = summrize_by_user('Test',test,cat_feat,num_feat,date_feat)
    
    val_pred,test_pred=lgb_kfold(train_X=trn_users,train_y=trn_y,test_X=test_users,n_splits=10)
"""    

"\ndef main():\n    cat_Feature=[]\n    num_feature=[]\n    \n    train = load_df('Train','input/train.csv')\n    test  = load_df('Test','input/test.csv')\n\n    train,date_feat = date_feature('Train',train)\n    test,_  = date_feature('Test',test)    \n    \n    train,cat_feat,num_feat = factorize_categorical('Train',train)\n    test,_,_  = factorize_categorical('Test',test)\n    \n    trn_users,trn_y = summrize_by_user('Train',train,cat_feat,num_feat,date_feat)\n    test_users,_    = summrize_by_user('Test',test,cat_feat,num_feat,date_feat)\n    \n    val_pred,test_pred=lgb_kfold(train_X=trn_users,train_y=trn_y,test_X=test_users,n_splits=10)\n"

In [5]:
%%time
cat_Feature=[]
num_feature=[]

train = load_df('train','../input/train.csv')
test  = load_df('test','../input/test.csv')

train,date_feat = date_feature('train',train)
test,_  = date_feature('test',test)    

train,cat_feat,num_feat,indexer = factorize_categorical('train',train,cat_indexer=None)
test,_,_,_  = factorize_categorical('test',test,cat_indexer=indexer)

#trn_users,trn_y = summrize_by_user('train',train,cat_feat,num_feat,date_feat)
#test_users,_    = summrize_by_user('test',test,cat_feat,num_feat,date_feat)

#val_pred,test_pred,feature_importance =lgb_kfold(train_X=trn_users,train_y=trn_y,test_X=test_users,n_splits=10)
#write_submission_file(test_X=test_users,test_pred=test_pred,filename='lgb_kfold')

#val_pred,test_pred,feature_importance =xgb_kfold(train_X=trn_users,train_y=trn_y,test_X=test_users,n_splits=10)
#write_submission_file(test_X=test_users,test_pred=test_pred,filename='xgb_kfold')

#display_feature_importance(feature_importance_df,modelName="Lightgbm Feature Importance (Over Folds)")
#display_feature_importance(feature_importance_df,modelName="XGBOOST Feature Importance (Over Folds)")

Loaded CSV For : train - Shape : (903653, 55)
Loaded CSV For : test - Shape : (804684, 54)
Date Feature Completed For: train - Shape:(903653, 61)
Date Feature Completed For: test - Shape:(804684, 60)
Factorize Completed For: train - Shape:(903653, 61)
Factorize Completed For: test - Shape:(804684, 60)
CPU times: user 6min 29s, sys: 14.5 s, total: 6min 43s
Wall time: 5min 28s


In [198]:
val_pred,test_pred,rf_feature_importance_df = rf_kfold(train_X=trn_users,train_y=trn_y,test_X=test_users,n_estimators=100,max_depth=6,verbose=2,n_splits=10)
val_pred,test_pred,feature_importance,lgb_validation_loss_df=lgb_kfold(train_X=trn_users,train_y=trn_y,test_X=test_users,learning_rate=0.01,n_estimators=20000,early_stopping_rounds=100,verbose=False,n_splits=10)

display_feature_importance(feature_importance_df_=feature_importance,modelName="LightGBM Feature Importance (Over Folds)")
display_loss(lgb_validation_loss_df,title='Lgbm Boosting Iteration Vs RMSE')

write_submission_file(test_X=test_users,test_pred=test_pred,filename='xgb_kfold')

if __name__ == "__main__":
    gc.enable
    main()
    print ("Model Completed..")

Train & Predict using Light GBM with Splits:10
Fold: 0   RMSE:1.6236771650239867   Best Iteration:639
Fold: 1   RMSE:1.5746025237166217   Best Iteration:499
Fold: 2   RMSE:1.5532708935327189   Best Iteration:661
Fold: 3   RMSE:1.582063240404973   Best Iteration:556
Fold: 4   RMSE:1.5871449289942965   Best Iteration:568
Fold: 5   RMSE:1.5676297658817453   Best Iteration:715
Fold: 6   RMSE:1.573098070782427   Best Iteration:598
Fold: 7   RMSE:1.5508281741586716   Best Iteration:638
Fold: 8   RMSE:1.5607572624316943   Best Iteration:673
Fold: 9   RMSE:1.5933419967428744   Best Iteration:620
Fold: A   RMSE:1.5765109076766548   Best Iteration:None


In [6]:
#---------------------------------
# Experiment from Here
#---------------------------------

In [7]:
train_y=train['totals.transactionRevenue'].fillna(0)

In [8]:
train_features = ['channelGrouping', 'visitNumber',
       'device.browser','device.deviceCategory','device.isMobile',
       'device.operatingSystem','geoNetwork.city','geoNetwork.continent',
       'geoNetwork.country','geoNetwork.metro','geoNetwork.networkDomain',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'trafficSource.adContent','trafficSource.campaign',
       'trafficSource.isTrueDirect','trafficSource.keyword',
       'trafficSource.medium','trafficSource.referralPath',
       'trafficSource.source']

In [9]:
%%time
val_pred,test_pred,lgb_feature_importance_df,lgb_validation_loss_df = lgb_kfold(train_X=train,train_y=np.log1p(train_y),test_X=test,features=train_features,verbose=False)

Train & Predict using Light GBM with Splits:10
Fold: 0   RMSE:1.6284244444344143   Best Iteration:563
Fold: 1   RMSE:1.6364796304531262   Best Iteration:551
Fold: 2   RMSE:1.625245765299165   Best Iteration:859
Fold: 3   RMSE:1.5856865953322508   Best Iteration:707
Fold: 4   RMSE:1.6852066514882296   Best Iteration:697
Fold: 5   RMSE:1.647896392597565   Best Iteration:698
Fold: 6   RMSE:1.6091558837339293   Best Iteration:767
Fold: 7   RMSE:1.63032083821995   Best Iteration:648
Fold: 8   RMSE:1.6431750661506315   Best Iteration:748
Fold: 9   RMSE:1.6206747349229396   Best Iteration:735
Fold: A   RMSE:7.873284945260392e+40   Best Iteration:None
CPU times: user 38min 16s, sys: 15min 39s, total: 53min 56s
Wall time: 12min 17s


In [8]:
train_X = train
train_y = np.log1p(train_y)
test_X = test
features=train_features

In [9]:
val_pred = np.zeros((train_X.shape[0]))
test_pred = np.zeros((test_X.shape[0]))

lgb_feature_importance_df = pd.DataFrame()
lgb_validation_loss_df = pd.DataFrame()

In [10]:
Folds = get_GroupKFolds(df=train_X, n_splits=10                      )
folds = Folds

In [12]:
params = {
    'learning_rate': 0.01,
    'n_estimators': 20000,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}



In [33]:
n_splits = 10
for fold_,(train_,val_) in enumerate(folds):
    lgb_model = lgb.LGBMRegressor(**params)

    lgb_model.fit(
        X=train_X[features].iloc[train_],
        y=train_y.iloc[train_],
        eval_set=[(train_X[features].iloc[train_],train_y.iloc[train_]),(train_X[features].iloc[val_],train_y.iloc[val_])],
        eval_metric='rmse',
        early_stopping_rounds=100,
        verbose=False
    )

    val_pred[val_]=lgb_model.predict(train_X[features].iloc[val_])
    curr_test_pred=lgb_model.predict(test_X[features])
    curr_test_pred[curr_test_pred < 0] = 0
    test_pred += curr_test_pred / n_splits
    print ("Fold: {}   RMSE:{}   Best Iteration:{}".format(fold_,mean_squared_error(val_pred[val_],train_y[val_])**0.5,lgb_model.best_iteration_) )

    fold_feature_importance_df = pd.DataFrame()
    fold_feature_importance_df["feature"] = train_X[features].columns
    fold_feature_importance_df["importance"] = lgb_model.feature_importances_
    fold_feature_importance_df["fold"] = fold_ + 1
    lgb_feature_importance_df = pd.concat([lgb_feature_importance_df,fold_feature_importance_df],axis=0)

    lgb_fold_validation_loss_df = pd.DataFrame()
    lgb_fold_validation_loss_df['rmse']= lgb_model.evals_result_['valid_1']['rmse']
    lgb_fold_validation_loss_df['fold'] = fold_+1
    lgb_validation_loss_df = pd.concat([lgb_fold_validation_loss_df,lgb_validation_loss_df],axis=0)


val_pred[val_pred<0] = 0

print ("Fold: A   RMSE:{}   Best Iteration:None".format(mean_squared_error(val_pred,train_y)**0.5) )
print ("Fold: A   RMSE:{}   Best Iteration:None".format(mean_squared_error(val_pred,train_y)**0.5) )

Fold: 0   RMSE:1.6284244444344143   Best Iteration:563
Fold: 1   RMSE:1.6364796304531262   Best Iteration:551
Fold: 2   RMSE:1.625245765299165   Best Iteration:859
Fold: 3   RMSE:1.5856865953322508   Best Iteration:707
Fold: 4   RMSE:1.6852066514882296   Best Iteration:697
Fold: 5   RMSE:1.647896392597565   Best Iteration:698
Fold: 6   RMSE:1.6091558837339293   Best Iteration:767
Fold: 7   RMSE:1.63032083821995   Best Iteration:648
Fold: 8   RMSE:1.6431750661506315   Best Iteration:748
Fold: 9   RMSE:1.6206747349229396   Best Iteration:735
Fold: A   RMSE:1.6311168247226826   Best Iteration:None
Fold: A   RMSE:1.6311168247226826   Best Iteration:None


In [26]:
print ("Fold: A   RMSE:{}   Best Iteration:None".format(mean_squared_error(val_pred,train_y)**0.5) )

Fold: A   RMSE:1.6311168247226826   Best Iteration:None
